# Recommender Systems with DGL

## Introduction

Graph Neural Networks (GNN), as a methodology of learning representations on graphs, has gained much attention recently.  Various models such as Graph Convolutional Networks, GraphSAGE, etc. are proposed to obtain representations of whole graphs, or nodes on a single graph.

A primary goal of Collaborative Filtering (CF) is to automatically make predictions about a user's interest, e.g. whether/how a user would interact with a set of items, given the interaction history of the user herself, as well as the histories of other users.  The user-item interaction can also be viewed as a bipartite graph, where users and items form two sets of nodes, and edges connecting them stands for interactions.  The problem can then be formulated as a *link-prediction* problem, where we try to predict whether an edge (of a given type) exists between two nodes.

Based on this intuition, the academia developed multiple new models for CF, including but not limited to:

* Geometric Learning Approaches
  * [Geometric Matrix Completion](https://papers.nips.cc/paper/5938-collaborative-filtering-with-graph-information-consistency-and-scalable-methods.pdf)
  * [Recurrent Multi-graph CNN](https://arxiv.org/pdf/1704.06803.pdf)
* Graph-convolutional Approaches
  * Models such as [R-GCN](https://arxiv.org/pdf/1703.06103.pdf) or [GraphSAGE](https://github.com/stellargraph/stellargraph/tree/develop/demos/link-prediction/hinsage) also apply.
  * [Graph Convolutional Matrix Completion](https://arxiv.org/abs/1706.02263)
  * [PinSage](https://arxiv.org/pdf/1806.01973.pdf)

## Dependencies

* Latest DGL release: `conda install -c dglteam dgl`
* `pandas`
* `stanfordnlp`
* `pytorch`
* `tqdm` for displaying the progress bar.

## Loading data

In this tutorial, we focus on rating prediction on MovieLens-1M dataset.  The data comes from [MovieLens](http://files.grouplens.org/datasets/movielens/ml-1m.zip) and is shipped with the notebook already.

After loading and train-validation-test-splitting the dataset, we process the movie title into (padded) word-ID sequences, and other features into categorical variables (i.e. integers).  We then store them as node features on the graph.

Since user features and item features are different, we pad both types of features with zeros.

All of the above is encapsulated in `movielens.MovieLens` class for clarity of this notebook.

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.function as FN

ImportError: dlopen(/usr/local/lib/python3.7/site-packages/torch/_C.cpython-37m-darwin.so, 9): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /usr/local/lib/python3.7/site-packages/torch/lib/libshm.dylib
  Reason: image not found

In [2]:
import movielens

ml = movielens.MovieLens('.')

ImportError: dlopen(/usr/local/lib/python3.7/site-packages/torch/_C.cpython-37m-darwin.so, 9): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /usr/local/lib/python3.7/site-packages/torch/lib/libshm.dylib
  Reason: image not found

## Model

We can now write a GraphSAGE layer.  In GraphSAGE, the node representation is updated with the representation in the previous layer as well as an aggregation (often mean) of "messages" sent from all neighboring nodes.

### Algorithm

The algorithm of a single GraphSAGE layer goes as follows for each node $v$:

1. $h_{\mathcal{N}(v)} \gets \mathtt{Average}_{u \in \mathcal{N}(v)} h_{u}$
2. $h_{v} \gets \sigma\left(W \cdot \mathtt{CONCAT}(h_v, h_{\mathcal{N}(v)})\right)$
3. $h_{v} \gets h_{v} / \lVert h_{v} \rVert_2$

where

* $\mathtt{Average}$ can be replaced by any kind of aggregation including `sum`, `max`, or even an LSTM.
* $\sigma$ is any non-linearity function (e.g. `LeakyReLU`)

We simply repeat the computation above for multiple GraphSAGE layers.

### DGL Message Passing

DGL adopts the message-passing paradigm, or scatter-apply-gather paradigm, for feature computation on a graph.  It decomposes the computation into three stages:

1. *Message computation*: each edge is computed a message according to features on the edge itself, as well as the features on its source and destination node.  Often times, the message computation simply involves copying the representation of the source node.
2. *Message aggregation*: each node then "receives" the messages sent from its neighbors, and call a function which reduces these messages into a single representation independent of the number of neighbors.  Averaging and summing are two of the most common message aggregation functions.
3. *Node feature update*: with an aggregated representation from the neighbors, a node then updates its own representation using the aggregation.

With the three stages in mind, we can easily figure out how to map the GraphSAGE layer computation into the message-passing paradigm:

1. $h_{\mathcal{N}(v)} \gets \underbrace{\mathtt{Average}_{u \in \mathcal{N}(v)} \underbrace{h_{u}}_{\text{Message computation (copy from source)}}}_{\text{Message aggregation}}$
2. $h_{v} \gets \underbrace{\sigma\left(W \cdot \mathtt{CONCAT}(h_v, h_{\mathcal{N}(v)})\right)}_{\text{Node feature update}}$
3. $h_{v} \gets \underbrace{h_{v} / \lVert h_{v} \rVert_2}_{\text{Node feature update}}$

While DGL does not provide the $\mathtt{Average}$ aggregation function yet (as it's a future work item), it does provide the $\mathtt{Sum}$ aggregation.  So we can modify the algorithm above to the following that is readily to be implemented in DGL:

1. $d_{\mathcal{N}(v)} \gets \underbrace{\mathtt{Sum}_{u \in \mathcal{N}(v)} \underbrace{1}_{\text{Message computation (copy from source)}}}_{\text{Message aggregation}}$
2. $h_{\mathcal{N}(v)} \gets \underbrace{\mathtt{Sum}_{u \in \mathcal{N}(v)} \underbrace{h_{u}}_{\text{Message computation (copy from source)}}}_{\text{Message aggregation}}$
3. $h_{v} \gets \underbrace{\sigma\left(W \cdot \mathtt{CONCAT}(h_v, h_{\mathcal{N}(v)} / d_{\mathcal{N}(v)})\right)}_{\text{Node feature update}}$
4. $h_{v} \gets \underbrace{h_{v} / \lVert h_{v} \rVert_2}_{\text{Node feature update}}$

In [4]:
def mix_embeddings(G, emb, proj):
    """Adds external (categorical and numeric) features into node representation G.ndata['h']"""
    extra_repr = []
    for key, scheme in G.node_attr_schemes().items():
        value = G.ndata[key]
        if scheme.dtype == torch.int64:
            result = self.emb[key](value)
            if result.dim() == 3:    # bag of words: the result would be a (n_nodes x seq_len x feature_size) tensor
                result = result.mean(1)
        elif scheme.dtype == torch.float32:
            result = self.proj[key](value)

        extra_repr.append(result)
    G.ndata['h'] = G.ndata['h'] + torch.stack(extra_repr, 0).sum(0)

class GraphSageConv(nn.Module):
    def __init__(self, feature_size):
        super(GraphSageConv, self).__init__()
        
        self.feature_size = feature_size

        self.W = nn.Linear(feature_size * 2, feature_size)

        init_weight(self.W.weight, 'xavier_uniform_', 'leaky_relu')
        init_bias(self.W.bias)

    def forward(self, nodes):
        h_agg = nodes.data['h_agg'] / nodes.data['w'][:, None]
        h = nodes.data['h_x']
        h_concat = troch.cat([h, h_agg, 1])
        h_new = F.leaky_relu(self.W(h_concat))
        return {'h': h_new / h_new.norm(1).clamp(min=1e-6)}
    
    
class GraphSage(nn.Module):
    def __init__(self, feature_size, num_layers, G):
        super(GraphSage, self).__init__()
        
        self.feature_size = feature_size
        self.num_layers = num_layers
        self.convs = nn.ModuleList([GraphSageConv(feature_size) for _ in range(num_layers)])
        
        self.G = G
        
        # For each categorical feature (including the word sequence) we create an embedding matrix.
        # For each numerical feature (e.g. the Genre vector, a binary vector indicating which
        # genre a movie belongs to) we create an affine layer.
        self.emb = nn.ModuleDict()
        self.proj = nn.ModuleDict()
        
        for key, scheme in G.node_attr_schemes().items():
            if scheme.dtype == torch.int64:
                self.emb[key] = nn.Embedding(G.ndata[key].max().item() + 1, feature_size, padding_idx=0)
            elif scheme.dtype == torch.float32:
                self.proj[key] = nn.Linear(scheme.shape[0], feature_size)
                
        self.node_emb = nn.Embedding(G.number_of_nodes(), feature_size)
        
    msg_funcs = [FN.copy_src('h', 'h'), FN.copy_src('one', 'one')]
    reduce_funcs = [FN.sum('h', 'h_agg'), FN.sum('one', 'w')]
        
    def forward(self):
        # Assign product- and user-specific embeddings first
        self.G.ndata['h'] = self.node_emb(torch.arange(self.G.number_of_nodes()))
        self.G.ndata['one'] = torch.ones(self.G.number_of_nodes())
        
        mix_embeddings(self.G, self.emb, self.proj)
        for i in range(self.num_layers):
            self.G.update_all(self.msg_funcs, self.reduce_funcs, self.convs[i])
        return self.G.ndata['h']

NameError: name 'dgl' is not defined

## Sampling

Ideally, we wish to execute a full update of the node embeddings with the GraphSAGE layer.  However, when the graph scales up, the full update soon becomes impractical, because the node embeddings couldn't fit in the GPU memory.

A natural solution would be partitioning the nodes and computing the embeddings one partition (minibatch) at a time.  The nodes at one convolution layer then only depends on their neighbors, rather than all the nodes in the graph, hence reducing the computational cost.  However, if we have multiple layers, and some of the nodes have a lot of neighbors (which is often the case since the degree distribution of many real-world graphs follow [power-law](https://en.wikipedia.org/wiki/Scale-free_network)), then the computation may still eventually depend on every node in the graph.

*Neighbor sampling* is an answer to further reduce the cost of computing node embeddings.  When aggregating messages, instead of collecting from all neighboring nodes, we only collect from some of the randomly-sampled (for instance, uniform sampling at most K neighbors without replacement) neighbors.

DGL provides the `NodeFlow` object that describes the computation dependency of nodes in a graph convolutional network, as well as various samplers that constructs such `NodeFlow`s as graphs.  From a programmer's perspective, training with minibatch and neighbor sampling reduces to propagating the messages in `NodeFlow` as follows.

In [6]:
class GraphSageConvWithSampling(nn.Module):
    def __init__(self, feature_size):
        super(GraphSageConv, self).__init__()

        self.feature_size = feature_size
        self.W = nn.Linear(feature_size * 2, feature_size)
        init_weight(self.W.weight, 'xavier_uniform_', 'leaky_relu')
        init_bias(self.W.bias)

    def forward(self, nodes):
        h_agg = nodes.data['h_agg']
        h = nodes.data['h']
        w = nodes.data['w'][:, None]
        # HACK 1:
        # When computing the representation of node v on layer L, we would like to
        # include the dependency of node v itself on layer L-1.  However, we don't
        # want to aggregate node v's own "message".  So we tell the sampler to
        # always "add self loop" to include such dependency, but we subtract the
        # node's representation from aggregation later.
        h_agg = safediv(h_agg - h, w - 1)    # HACK 1
        h_concat = torch.cat([h, h_agg], 1)
        h_new = F.leaky_relu(self.W(h_concat))
        return {'h': safediv(h_new, h_new.norm(dim=1, keepdim=True))}
    
class GraphSageWithSampling(nn.Module):
    def __init__(self, feature_size, n_layers, G):
        super(GraphSage, self).__init__()
        
        self.feature_size = feature_size
        self.n_layers = n_layers

        self.convs = nn.ModuleList([GraphSageConv(feature_size) for _ in range(n_layers)])
        
        self.emb = nn.ModuleDict()
        self.proj = nn.ModuleDict()

        for key, scheme in G.node_attr_schemes().items():
            if scheme.dtype == torch.int64:
                self.emb[key] = emb.get(key, ScaledEmbedding(
                        G.ndata[key].max().item() + 1,
                        self.in_features,
                        padding_idx=0))
            elif scheme.dtype == torch.float32:
                w = nn.Linear(scheme.shape[0], self.in_features)
                init_weight(w.weight, 'xavier_uniform_', 'leaky_relu')
                init_bias(w.bias)
                self.proj[key] = proj.get(key, nn.Sequential(w, nn.LeakyReLU()))
                
        self.G = G
        
        self.node_emb = nn.Embedding(G.number_of_nodes(), feature_size)

    msg = [FN.copy_src('h', 'h'),
           FN.copy_src('one', 'one')]
    red = [FN.sum('h', 'h_agg'), FN.sum('one', 'w')]

    def forward(self, nf, h_emb):
        '''
        nf: NodeFlow.
        '''
        # Assign product- and user-specific embeddings first
        self.G.ndata['h'] = self.node_emb(torch.arange(self.G.number_of_nodes()))
        self.G.ndata['one'] = torch.ones(self.G.number_of_nodes())
        
        mix_embeddings(self.G, self.emb, self.proj)
        nf.copy_from_parent(edge_embed_names=None)

        for i in range(nf.num_blocks):
            nf.block_compute(i, self.msg, self.red, self.convs[i])

        result = nf.layers[-1].data['h']
        assert (result != result).sum() == 0
        return result

g = ml.g
# Find the subgraph of all "training" edges
g_train = g.edge_subgraph(g.filter_edges(lambda edges: edges.data['train']), True)
src, dst = g_train.all_edges()

batch_size = 1024

for epoch in range(50):
    shuffle_idx = torch.randperm(shuffle_idx)
    src_shuffled = src[shuffle_idx]
    dst_shuffled = dst[shuffle_idx]
    src_batches = src.split(batch_size)
    dst_batches = dst.split(batch_size)
    
    # HACK 2: Alternate between source batch and destination batch, so we can put exactly
    # a batch of edges' endpoints in a single NodeFlow.
    seed_nodes = sum([[s, d] for s, d in zip(src_batches, dst_batches)], [])
    
    sampler = NeighborSampler(
            g_train,
            batch_size * 2,        # HACK 2
            5,
            n_layers,
            seed_nodes=seed_nodes, # HACK 2
            prefetch=True,
            add_self_loop=True,    # HACK 1
            shuffle=False,
            num_workers=4)

NameError: name 'FN' is not defined

## Training

As above, training now only involves
1. Initializing a sampler
2. Iterating over the neighbor sampler, propagating the messages, and computing losses and gradients as usual.

Meanwhile, we also evaluate the RMSE on validation and test set.